In [ ]:
import os
import torch
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

os.environ["CUDA_VISIBLE_DEVICES"] = "4"
print(torch.cuda.device_count())  # 输出应该是 1，代表只看到 GPU 2
print(torch.cuda.current_device())  # 检查当前使用的 GPU
print(torch.cuda.get_device_name())  # 检查正在使用哪个 GPU

from final_LLM_enhance.LLM_process_complex import analyze_fine_moton_control_txt
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

from exit.utils import animate3d, t2m_bone
import numpy as nps
import torch
from tqdm import tqdm
from exit.utils import visualize_2motions
from utils.motion_process import recover_from_ric
from generate import MMM
from dataset import dataset_TM_eval
from utils.word_vectorizer import WordVectorizer

w_vectorizer = WordVectorizer('/home/haoyum3/MMM/glove/', 'our_vab')
val_loader = dataset_TM_eval.DATALoader('t2m', True, 32, w_vectorizer)
print("1")
class Temp:
    def __init__(self, extra_args=None):
        print('mock:: opt')
        if extra_args is not None:
            for i in extra_args:
                self.__dict__[i] = extra_args[i]

args = Temp()

# 只修改已有的变量
args.dataname = args.dataset_name = 't2m'
args.nb_code = 256
args.code_dim = 32
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.quantizer = 'ema_reset'
args.mu = 0.99
args.clip_dim = 512

# 修改 embed_dim_gpt, block_size, num_layers, n_head_gpt
args.embed_dim_gpt = 256  # 从 512 修改为 256
args.block_size = 51  # 保持不变
args.num_layers = 6  # 从 9 修改为 6
args.n_head_gpt = 8  # 从 16 修改为 8
args.drop_out_rate = 0.1  # 保持不变
args.ff_rate = 4  # 保持不变
args.resume_pth = '/home/haoyum3/MMM/output/vq/2024-08-19-00-17-48_vq_general_decoder/net_last.pth'
# args.resume_pth = '/home/haoyum3/MMM/output/vq/256_6_8_recaption/net_last.pth'
args.resume_trans = '/home/haoyum3/MMM/output/t2m/256_6_8_recaption/net_best_fid.pth' 

# 保持 num_local_layer 不变
args.num_local_layer = 0

# 保留 retrieve 调用部分
from final_LLM_enhance.retrieval_based_solution import retrieve
retrieve("A bodybuilder performs a perfect pistol squat.")

# 假设 val_loader 和 MMM 在其他地方定义
args.mean = val_loader.dataset.mean
args.std = val_loader.dataset.std
mmm = MMM(args, is_time=True).cuda()




In [ ]:


# 定义要遍历的目录路径
directory = "/home/haoyum3/MMM/llm/zsh/output3/new_retrieve_or_not_npy"

# 遍历目录中的所有文件
for filename in os.listdir(directory):
    if filename.endswith(".npy"):  # 只处理 .npy 文件
        # 获取文件的完整路径
        filepath = os.path.join(directory, filename)

        # 初始化 flag
        flag = None

        # 根据前缀设置 flag，并去掉前缀
        if filename.startswith("retrieved_"):
            flag = "retrieved"
            new_name = filename.replace("retrieved_", "").replace("_", " ").replace(".npy", "")
        elif filename.startswith("no_action_"):
            flag = "no_action"
            new_name = filename.replace("no_action_", "").replace("_", " ").replace(".npy", "")
        else:
            new_name = filename.replace("_", " ").replace(".npy", "")

        # 输出旧文件名、新文件名和标志
        print(f"Old name: {filename} -> New name: {new_name}, Flag: {flag}")

        # 重命名文件（确保无误后取消注释以下行）
        # os.rename(filepath, os.path.join(directory, new_name + ".npy"))


In [ ]:
import clip
##### ---- CLIP ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

# https://github.com/openai/CLIP/issues/111
class TextCLIP(torch.nn.Module):
    def __init__(self, model) :
        super(TextCLIP, self).__init__()
        self.model = model
        
    def forward(self,text):
        with torch.no_grad():
            word_emb = self.model.token_embedding(text).type(self.model.dtype)
            word_emb = word_emb + self.model.positional_embedding.type(self.model.dtype)
            word_emb = word_emb.permute(1, 0, 2)  # NLD -> LND
            word_emb = self.model.transformer(word_emb)
            word_emb = self.model.ln_final(word_emb).permute(1, 0, 2).float()
            enctxt = self.model.encode_text(text).float()
        return enctxt, word_emb
clip_model = TextCLIP(clip_model)



In [ ]:
import os

npy_dir_root = "/home/haoyum3/MMM/llm/zsh/output4/finemotiontest_npy/"

# Check if the directory exists, if not, create it
if not os.path.exists(npy_dir_root):
    os.makedirs(npy_dir_root)
    print(f"Directory {npy_dir_root} created.")
else:
    print(f"Directory {npy_dir_root} already exists.")
    

In [ ]:
# Define the body part indices
BODY_PARTS = {
    'left_arm': 0,
    'right_arm': 1,
    'left_leg': 2,
    'right_leg': 3,
    'spine': 4
}

# Define modes with masking percentages for each body part
modes = {
    'modify_upper_body': {
        BODY_PARTS['left_arm']: 0.8,
        BODY_PARTS['right_arm']: 0.8,
        BODY_PARTS['left_leg']: 0.2,
        BODY_PARTS['right_leg']: 0.2,
        BODY_PARTS['spine']: 0.2
    },
    'modify_left_arm': {
        BODY_PARTS['left_arm']: 1.0,
        BODY_PARTS['right_arm']: 0.0,
        BODY_PARTS['left_leg']: 0.0,
        BODY_PARTS['right_leg']: 0.0,
        BODY_PARTS['spine']: 0.0
    },
    'modify_upper_body1': {
        BODY_PARTS['left_arm']: 1.0,
        BODY_PARTS['right_arm']: 0.6,
        BODY_PARTS['left_leg']: 0.0,
        BODY_PARTS['right_leg']: 0.0,
        BODY_PARTS['spine']: 0.0,
    },
    'modify_upper_body2': {
        BODY_PARTS['left_arm']: 0.8,
        BODY_PARTS['right_arm']: 0.0,
        BODY_PARTS['left_leg']: 0.0,
        BODY_PARTS['right_leg']: 0.0,
        BODY_PARTS['spine']: 0.0,
    },
    # Add more modes as needed
}

# Select the desired mode
selected_mode = modes['modify_upper_body2']  # Change to the desired mode

# The rest of your code remains largely the same, with modifications in the masking section
textA = ['a person walks clockwise']
texts = [
    'A person waves both hands greatly above head',
    'A person waves both hands greatly above head',
    'A guy launches into a powerful kick',
    'A guy launches into a powerful kick',
    'A guy launches into a powerful kick'
]
m_length = torch.tensor([120])
motion2 = mmm(textA, m_length.cuda(), rand_pos=True)

# New motion length
m_length = torch.tensor([196])

# Begin expanding multi_edit logic
pose = motion2.clone().cuda().float()  # Clone and convert to float
m_tokens_len = torch.ceil((m_length) / 4).int()  # Downsample, compute motion token length
bs, seq = pose.shape[:2]  # Batch size and pose sequence length
max_motion_length = int(seq / 4) + 1  # Max motion length (+1 for end token)
mot_end_idx = mmm.vqvae.vqvae.num_code  # Motion end token index
mot_pad_idx = mmm.vqvae.vqvae.num_code + 1  # Padding token index
mask_id = mmm.vqvae.vqvae.num_code + 2  # Mask token index

# Target token list
target_tokens = []
for k in range(bs):
    # Encode current sample's pose
    target = mmm.vqvae.vqvae.teacher_net(pose[k:k+1, :m_length[k]], type='encode')

    # Pad if necessary
    if m_tokens_len[k] + 1 < max_motion_length:
        target = torch.cat([
            target,
            torch.ones((1, 1, 5), dtype=int, device=target.device) * mot_end_idx,
            torch.ones((1, max_motion_length - 1 - m_tokens_len[k].item(), 5), dtype=int, device=target.device) * mot_pad_idx
        ], axis=1)
    else:
        target = torch.cat([
            target,
            torch.ones((1, 1, 5), dtype=int, device=target.device) * mot_end_idx
        ], axis=1)

    target_tokens.append(target)

# Concatenate all samples' target tokens and adjust dimensions
target_tokens = torch.cat(target_tokens, axis=0).permute(0, 2, 1)  # Shape: [bs, 5, max_motion_length]

### New masking logic ###
target_masked = target_tokens.clone()  # Clone target tokens for masking

# Initialize mask
mask = torch.zeros(bs, 5, max_motion_length, dtype=torch.bool, device=target_tokens.device)

for k in range(bs):
    mask_len = m_tokens_len[k].item() + 1  # Include end token
    for idx in range(5):  # Loop over body part indices
        p = selected_mode.get(idx, 0.0)  # Get masking percentage, default to 0.0 if not specified
        # Generate mask for the valid token positions (up to mask_len)
        rand_vals = torch.rand(mask_len, device=target_tokens.device)
        body_part_mask = rand_vals < p
        mask[k, idx, :mask_len] = body_part_mask
    # Ensure end tokens are not masked
    end_token_pos = mask_len - 1  # Position of end token
    mask[k, :, end_token_pos] = False  # Do not mask end token

# Apply mask
target_masked[mask] = mask_id

# Ensure that no invalid token indices are present
vocab_size = mmm.vqvae.vqvae.num_code + 3  # Adjust based on your actual vocab size
if target_masked.max() >= vocab_size:
    print(f"Invalid token index detected: {target_masked.max()}")
    target_masked = torch.clamp(target_masked, max=vocab_size - 1)

# Initialize prediction data
pred_len = m_length.cuda()
pred_tok_len = m_tokens_len
pred_pose_eval = torch.zeros((bs, seq, pose.shape[-1])).cuda()

### Text feature processing ###
feat_clip_texts = []
for text in texts:
    text = clip.tokenize(text, truncate=True).cuda()  # Process text
    feat_clip_text, word_emb_clip = clip_model(text)  # Extract text features
    feat_clip_texts.append(feat_clip_text)

feat_clip_texts = torch.stack(feat_clip_texts, dim=1)  # Stack text features

mask

In [ ]:

### 文本特征处理 ###
feat_clip_texts = []
for text in texts:
    text = clip.tokenize(text, truncate=True).cuda()  # 处理文本
    feat_clip_text, word_emb_clip = clip_model(text)  # 提取文本特征
    feat_clip_texts.append(feat_clip_text)
len(feat_clip_texts)#5
feat_clip_texts = torch.stack(feat_clip_texts, dim=1)  # 堆叠文本特征
feat_clip_texts.shape
feat_clip_texts


In [ ]:
edit_idx = torch.tensor([0], dtype=torch.long)

index_motion = mmm.maskdecoder(
    target_masked,
    feat_clip_texts,
    word_emb_clip,
    edit_idxs=edit_idx,
    m_length=pred_len,
    rand_pos=True,
    type="inpaint"
)
index_motion


In [ ]:
# 解码生成的动作序列并将其存储到 pred_pose_eval
for i in range(bs):
    all_tokens = index_motion[i:i+1, :int(pred_tok_len[i].item())]
    pred_pose = mmm.vqvae.vqvae.teacher_net(all_tokens, type='decode')
    pred_pose_eval[i:i+1, :int(pred_len[i].item())] = pred_pose

# 从预测结果中提取姿势数据并进行后处理
pose = pred_pose_eval[0, :].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
pose = recover_from_ric(torch.from_numpy(pose).float(), 22).numpy()

# 使用 animate3d 动画展示结果
animate3d(pose, BONE_LINK=t2m_bone, first_total_standard=63, axis_visible=True)

# 保存结果
filename = f"1.npy"
save_path = os.path.join(npy_dir_root, filename)
import numpy as np
np.save(save_path, pose)
print(f"Motion data saved to {save_path}")